In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix, accuracy_score

# ----------------------------------------------
# Cargar el dataset (Asegúrate de subir el archivo CSV a Colab)
df = pd.read_csv("train.csv")
display(df.head())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [46]:
# 2. PREPROCESAMIENTO DE LOS DATOS
print("Eliminando valores nulos...")
df.dropna(inplace=True)

# Crear la columna PriceCategory si no existe
def categorize_price(price):
    if price < 130000:
        return "Barata"
    elif price < 250000:
        return "Media"
    else:
        return "Cara"

print("Creando columna PriceCategory basada en SalePrice...")
df["PriceCategory"] = df["SalePrice"].apply(categorize_price)

# Definir variables predictoras y variables respuesta
print("Seleccionando características y variables de salida...")
X = df[['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']]
y_reg = df["SalePrice"]  # Para regresión
y_clf = df["PriceCategory"]  # Para clasificación

# Codificar variable categórica para clasificación
print("Codificando la variable PriceCategory...")
le = LabelEncoder()
y_clf = le.fit_transform(y_clf)

# Verificar que hay datos suficientes
if len(df) > 0:
    print("Dividiendo en conjuntos de entrenamiento y prueba...")
    X_train, X_test, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)
    X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y_clf, test_size=0.2, random_state=42)

    # Normalización de los datos
    print("Normalizando los datos...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)  # Normalización para regresión
    X_test_scaled = scaler.transform(X_test)  # Normalización para regresión
    X_train_clf_scaled = scaler.fit_transform(X_train_clf)  # Normalización para clasificación
    X_test_clf_scaled = scaler.transform(X_test_clf)  # Normalización para clasificación



Eliminando valores nulos...
Creando columna PriceCategory basada en SalePrice...
Seleccionando características y variables de salida...
Codificando la variable PriceCategory...


In [68]:

# 3. MODELO DE REGRESIÓN CON KNN
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train_scaled, y_train_reg)  # Entrenando el modelo de regresión
y_pred_knn = knn_reg.predict(X_test_scaled)

# Evaluación del modelo de regresión
mse_knn = mean_squared_error(y_test_reg, y_pred_knn)
r2_knn = r2_score(y_test_reg, y_pred_knn)
print(f"KNN Regresión - MSE: {mse_knn:.2f}, R²: {r2_knn:.2f}")


NameError: name 'X_train_scaled' is not defined

In [ ]:
# 4. COMPARACIÓN CON OTROS MODELOS DE REGRESIÓN
models = {"Regresión Lineal": LinearRegression(),
          "Árbol de Regresión": DecisionTreeRegressor(),
          "Naive Bayes": GaussianNB()}

for name, model in models.items():
    model.fit(X_train_scaled, y_train_reg)  # Usando X_train_scaled para entrenar
    y_pred = model.predict(X_test_scaled)  # Predicciones
    mse = mean_squared_error(y_test_reg, y_pred)  # Evaluación de MSE
    r2 = r2_score(y_test_reg, y_pred)  # Evaluación de R²
    print(f"{name} - MSE: {mse:.2f}, R²: {r2:.2f}")


In [ ]:
# 5. MODELO DE CLASIFICACIÓN CON KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_clf_scaled, y_train_clf)  # Entrenando el modelo de clasificación
y_pred_clf = knn_clf.predict(X_test_clf_scaled)

# Evaluación de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test_clf, y_pred_clf, target_names=le.classes_))


In [ ]:
# 6. MATRIZ DE CONFUSIÓN
cm = confusion_matrix(y_test_clf, y_pred_clf)
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - KNN Clasificación")
plt.show()


In [ ]:
# 7. VALIDACIÓN CRUZADA PARA REGRESIÓN Y CLASIFICACIÓN
cv_scores_reg = cross_val_score(knn_reg, X_train_scaled, y_train_reg, cv=5, scoring='r2')
cv_scores_clf = cross_val_score(knn_clf, X_train_clf_scaled, y_train_clf, cv=5, scoring='accuracy')

print(f"Precisión promedio en validación cruzada (Clasificación): {cv_scores_clf.mean():.2f}")
print(f"R² promedio en validación cruzada (Regresión): {cv_scores_reg.mean():.2f}")


In [ ]:
# 8. CONCLUSIÓN SOBRE SOBREAJUSTE
if r2_knn > max([r2_score(y_test_reg, models[m].predict(X_test_scaled)) for m in models]):
    print("El modelo KNN parece estar sobreajustado en comparación con otros modelos.")
else:
    print("El modelo KNN no muestra signos evidentes de sobreajuste.")
